In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np
import dill
%matplotlib inline

In [ ]:
# Set up global matplotlib params. 
matplotlib.rc('font',family='serif')
matplotlib.rc('text', usetex=True)
# matplotlib.rc('font',family='serif', serif=['Times'])

# Two Player Zero Sum

In [ ]:
# Load the data.
path_to_log = '../python/logs/two_player_zero_sum/unicycle_4d_example.pkl'
fp = open(path_to_log, 'rb')
data = dill.load(fp)
fp.close()

In [ ]:
# Environment parameters.
goal = (100.0, 100.0)
obstacle_centers = [(100.0, 35.0), (65.0, 65.0), (25.0, 80.0)]
obstacle_radii = [10.0, 10.0, 10.0]
x_lims = (0, 150)
y_lims = (0, 125)

In [ ]:
def plot_obstacles(ax):
    for center, radius in zip(obstacle_centers, obstacle_radii):
        circle = plt.Circle((center[0], center[1]), radius, color='grey', fill=True, alpha=0.75)
        ax.add_artist(circle)
        # ax.text(center[0] - 1.25, center[1] - 1.25, "obs", fontsize=8)
        
def plot_goal(ax):
    ax.scatter(goal[0], goal[1], s=75, c='g', marker='*')

In [ ]:
def plot_vel_traj(ax, traj, downsample, vel_lims=None):    
    xs = np.array([x[0, 0] for k, x in enumerate(traj) if k % downsample == 0])
    ys = np.array([x[1, 0] for k, x in enumerate(traj) if k % downsample == 0])
    
    # Use the velocity along the trajectory to color it.
    vel = np.array([x[3, 0] for k, x in enumerate(traj) if k % downsample == 0])
    
    points = np.array([xs, ys]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)    
    
    min_vel = vel_lims[0] if vel_lims is not None else vel.min()
    max_vel = vel_lims[1] if vel_lims is not None else vel.max()
    
    print("min_vel = {}, max_vel = {}".format(min_vel, max_vel))
    
    norm = plt.Normalize(min_vel, max_vel)
    lc = LineCollection(segments, cmap='viridis', norm=norm)

    # Set the velocity to be used for color mapping.
    lc.set_array(vel)
    lc.set_linewidth(2)
    line = ax.add_collection(lc)
    
    return line

In [ ]:
def plot_iteration(ax, iteration, downsample=1):
    ax.set_title('Iteration ' + str(iteration), fontsize=18)
    
    # Plot the obstacles.
    plot_obstacles(ax)
    
    # Plot the goal.
    plot_goal(ax)
    
    # Plot the trajectory.
    traj = data['xs'][iteration]
    
    xs = [x[0, 0] for k, x in enumerate(traj) if k % downsample == 0]
    ys = [x[1, 0] for k, x in enumerate(traj) if k % downsample == 0]
    
    traj_line = None
    traj_line = plot_vel_traj(ax, traj, downsample, vel_lims=(0, 30))
#     ax.plot(xs, ys, '.-b', label="ILQ game", alpha=1., markersize=2)

    ax.set_xlim(x_lims)
    ax.set_ylim(y_lims)
    ax.set_xlabel('$x(t)$', fontsize=16)
    ax.set_ylabel('$y(t)$', fontsize=16)

    # Remove the x and y ticks.
    plt.sca(ax)
    plt.xticks([])
    plt.yticks([])
    
    return traj_line

In [ ]:
iterations = [0, 50, 100, 150]
fig, axes = plt.subplots(1, len(iterations))

traj_line = None
for i, axi in zip(iterations, axes):
    traj_line = plot_iteration(axi, i, downsample=3)
        
if traj_line is not None:
#     fig.colorbar(traj_line, ax=axes[-1])
    fig.colorbar(traj_line, ax=axes.ravel().tolist())

fig.set_figwidth(4 * len(iterations) + 2)
fig.set_figheight(3.4)
plt.savefig('plots/two_player_zero_sum_top.pdf', bbox_inches='tight')